<a href="https://colab.research.google.com/github/francisconetodata/exemplos-jupyter-blog/blob/main/02_simples_bancodedados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial de como criar um banco de dados simples pelo Python

(Usando o Google Colab)

Francisco de Assis Pereira Neto - [Linkedin](https://www.linkedin.com/in/francisconetomaq/)


Idea deste tutorial: Criar um banco de dados sqlite3 usando apenas o Colab para inserir e consultar dados de vazão de um Rio fictício.


(Apenas um primeiro script simples e que nos próximos serão feitos exemplos mais elaborados).

O primeiro passo é importar o pacote e criar um banco de dados com os seguintes comandos:

In [1]:
import sqlite3 # Pacote do banco de dados
conn = sqlite3.connect('monitorar.db') # Criando o banco de dados.
conn.close() # Fechando conexão com o banco de dados.

In [2]:
# Gerador de números aleatórios para criar vazão fictícia:
# Serão gerados números de uma série uniforme (com intervalos entre 3 e 7)
# Round(x,2) arredonda o número para duas casas decimais.
import random
import time # Função apenas para colocar medições temporizadas.
x = round(random.uniform(a=3,b=7),2)
print(x)

4.8


In [3]:
import sqlite3
# Conectar com o banco de dados
conn = sqlite3.connect('monitorar.db')
# Definindo um cursos para executar as funções.
cursor = conn.cursor()

# Criando tabela vazao
# Comentário em SQL (depois de --)
cursor.execute("""
CREATE TABLE vazao (
        ID INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,  -- ID do registro
        NomeRio TEXT NOT NULL,                          -- Nome do rio monitorado
        Criado TIMESTAMP NULL DEFAULT CURRENT_TIMESTAMP,-- Momento do registro (automático)
        Cidade TEXT NOT NULL,                           -- Cidade do Rio 
        UF VARCHAR(2) NOT NULL,                         -- Estados do Rio 
        Vazao REAL NOT NULL                             -- Vazão do Rio
);
""")
print('Comando executado com sucesso !!!')
# desconectando...
conn.close()

Comando executado com sucesso !!!


In [4]:
import sqlite3
conn = sqlite3.connect('monitorar.db')
cursor = conn.cursor()

In [5]:
# Existem diversas formas de inserir dados
# Busquei fazer algo aleatório apenas para construir o Banco de dados.
nome_rios = ['Salinas',
             'São Francisco',
             'Tietê',
             'Lavras',
             'Amazonas']
uf_rios = ['MG',
           'MG',
           'SP',
           'MG',
           'AM']
cidades_rios = ['Salinas',
                'Janaúba',
                'São Paulo',
                'Lavras',
                'Manaus']

for i in range(1500):
    time.sleep(0.1) # Tempo de espera de 1/10 de segundo.
    for j in range(5):
        x = round(random.uniform(a=3,b=7),2)
        cursor.execute(f"""
        INSERT INTO vazao (NomeRio, Cidade, UF, Vazao)
        VALUES ('{nome_rios[j]}','{cidades_rios[j]}','{uf_rios[j]}','{x}')
        """)
        
# Gravando no banco de dados:
conn.commit()

In [9]:
# Leitura dos dados (apenas os 5 primeiros do banco de dados):
cursor.execute("""
SELECT * FROM vazao
WHERE vazao.ID < 6;
""")
for linha in cursor.fetchall():
    print(linha)

(1, 'Salinas', '2022-03-07 03:39:26', 'Salinas', 'MG', 4.29)
(2, 'São Francisco', '2022-03-07 03:39:26', 'Janaúba', 'MG', 4.68)
(3, 'Tietê', '2022-03-07 03:39:26', 'São Paulo', 'SP', 6.85)
(4, 'Lavras', '2022-03-07 03:39:26', 'Lavras', 'MG', 5.0)
(5, 'Amazonas', '2022-03-07 03:39:26', 'Manaus', 'AM', 3.24)


In [10]:
# Lendo dados específicos:
# Pergunta: Qual a vazão média dos rios de MG ?
cursor.execute("""
SELECT v.UF, AVG(v.Vazao) AS VazaoMedia FROM vazao v
WHERE v.UF LIKE 'MG'
GROUP BY v.UF;
""")
for linha in cursor.fetchall():
    print(linha)

('MG', 4.990724444444432)


In [11]:
# Pergunta: Qual a vazão média de todos os rios (por Estado) ?
cursor.execute("""
SELECT v.UF, AVG(v.Vazao) AS VazaoMedia FROM vazao v
GROUP BY v.UF;
""")
for linha in cursor.fetchall():
    print(linha)

('AM', 4.968513333333334)
('MG', 4.990724444444432)
('SP', 4.991046666666675)


In [12]:
# Pergunta: Qual a maior e menor vazão por Rio ?
cursor.execute("""
SELECT v.NomeRio, MAX(v.Vazao), MIN(v.Vazao) FROM vazao v
GROUP BY v.NomeRio;
""")
for linha in cursor.fetchall():
    print(linha)

('Amazonas', 7.0, 3.0)
('Lavras', 7.0, 3.0)
('Salinas', 7.0, 3.0)
('São Francisco', 7.0, 3.0)
('Tietê', 6.99, 3.0)


In [13]:
# Pergunta: Qual a vazão média de cada rio ?
cursor.execute("""
SELECT v.NomeRio, AVG(v.Vazao) FROM vazao v
GROUP BY v.NomeRio;
""")
for linha in cursor.fetchall():
    print(linha)

('Amazonas', 4.968513333333334)
('Lavras', 5.015440000000002)
('Salinas', 4.970966666666666)
('São Francisco', 4.985766666666665)
('Tietê', 4.991046666666675)


In [14]:
# Pergunta: Quantas vezes cada rio passou a vazão maior ou igual a 6.5 ?
cursor.execute("""
SELECT v.NomeRio, COUNT(DISTINCT(v.ID)) FROM vazao v
WHERE v.Vazao >= 6.5
GROUP BY v.NomeRio
;
""")
for linha in cursor.fetchall():
    print(linha)

('Amazonas', 175)
('Lavras', 206)
('Salinas', 162)
('São Francisco', 188)
('Tietê', 189)
